In [1]:
from pyspark.sql import SparkSession
spark = SparkSession \
            .builder \
            .master('spark://vmurashkin-osx2.local:7077') \
            .appName('SQL App').getOrCreate()
sc = spark.sparkContext

In [2]:
!cat ../data/sales_header.csv

Transaction_date,Product,Price,Payment_Type,Name,City,State,Country,Account_Created,Last_Login,Latitude,Longitude


In [3]:
!head ../data/sales.csv

1/2/09 6:17,Product1,1200,Mastercard,carolina,Basildon,England,United Kingdom,1/2/09 6:00,1/2/09 6:08,51.5,-1.1166667
1/2/09 4:53,Product1,1200,Visa,Betina,Parkville                   ,MO,United States,1/2/09 4:42,1/2/09 7:49,39.195,-94.68194
1/2/09 13:08,Product1,1200,Mastercard,Federica e Andrea,Astoria                     ,OR,United States,1/1/09 16:21,1/3/09 12:32,46.18806,-123.83
1/3/09 14:44,Product1,1200,Visa,Gouya,Echuca,Victoria,Australia,9/25/05 21:13,1/3/09 14:22,-36.1333333,144.75
1/4/09 12:56,Product2,3600,Visa,Gerd W ,Cahaba Heights              ,AL,United States,11/15/08 15:47,1/4/09 12:45,33.52056,-86.8025
1/4/09 13:19,Product1,1200,Visa,LAURENCE,Mickleton                   ,NJ,United States,9/24/08 15:19,1/4/09 13:04,39.79,-75.23806
1/4/09 20:11,Product1,1200,Mastercard,Fleur,Peoria                      ,IL,United States,1/3/09 9:38,1/4/09 19:45,40.69361,-89.58889
1/2/09 20:09,Product1,1200,Mastercard,adam,Martin                      ,TN,United States,1/2/09 17:43,1/4/

In [2]:
def parse_row(line):
    return line.split(',')

sales_rdd = sc.textFile('../data/sales.csv').map(parse_row)

In [5]:
print(sales_rdd.take(3))

[['1/2/09 6:17', 'Product1', '1200', 'Mastercard', 'carolina', 'Basildon', 'England', 'United Kingdom', '1/2/09 6:00', '1/2/09 6:08', '51.5', '-1.1166667'], ['1/2/09 4:53', 'Product1', '1200', 'Visa', 'Betina', 'Parkville                   ', 'MO', 'United States', '1/2/09 4:42', '1/2/09 7:49', '39.195', '-94.68194'], ['1/2/09 13:08', 'Product1', '1200', 'Mastercard', 'Federica e Andrea', 'Astoria                     ', 'OR', 'United States', '1/1/09 16:21', '1/3/09 12:32', '46.18806', '-123.83']]


In [4]:
with open('../data/sales_header.csv') as src:
    sales_header = src.read().strip().split(',')

sales_df = sales_rdd.toDF(sales_header)
sales_df.registerTempTable('sales')

#print(sales_total_df.rdd.collect())

# TODO: print revenue by Country and State

In [5]:
sales_total_df = spark.sql(
    "SELECT Country, State, SUM(Price) AS Revenue \
     FROM sales \
     GROUP BY Country, State \
     ORDER BY Revenue DESC LIMIT 10")

print(sales_total_df.collect())

[Row(Country='United Kingdom', State='England', Revenue=120000.0), Row(Country='United States', State='CA', Revenue=113350.0), Row(Country='United States', State='NY', Revenue=61200.0), Row(Country='United States', State='TX', Revenue=55500.0), Row(Country='United States', State='FL', Revenue=51600.0), Row(Country='Canada', State='Ontario', Revenue=46800.0), Row(Country='United States', State='VA', Revenue=40400.0), Row(Country='Ireland', State='Dublin', Revenue=28800.0), Row(Country='Canada', State='British Columbia', Revenue=28800.0), Row(Country='United States', State='GA', Revenue=28200.0)]


In [ ]:
print(sales_df.rdd.take(3))

In [20]:
print(sales_df.rdd.map(lambda row: 
                       ((row.Country, row.State), 
                        float(row.Price)))\
                .reduceByKey(lambda a, b: a + b)\
                .top(10, key=lambda e: e[1]))

[(('United Kingdom', 'England'), 120000.0), (('United States', 'CA'), 113350.0), (('United States', 'NY'), 61200.0), (('United States', 'TX'), 55500.0), (('United States', 'FL'), 51600.0), (('Canada', 'Ontario'), 46800.0), (('United States', 'VA'), 40400.0), (('Ireland', 'Dublin'), 28800.0), (('Canada', 'British Columbia'), 28800.0), (('United States', 'GA'), 28200.0)]
